In [0]:
#Download and unzip files
!pip3 install scikit-learn
!sudo apt-get install unzip
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torch torchvision
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
     |████████████████████████████████| 614.8MB 28kB/s 
ERROR: torchvision 0.6.0+cu101 has requirement torch==1.5.0, but you'll have torch 1.0.1 which is incompatible.
  Found existing installation: torch 1.5.0+cu101
    Uninstalling torch-1.5.0+cu101:
      Successfully uninstalled torch-1.5.0+cu101
--2020-05-04 04:17:21--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.85.37
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.85.37|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  36.6MB/s    in 44s   

In [0]:
#!unzip 'MS4-20200502T223625Z-001.zip'

In [0]:
#Verfiy file download
#models.py
import torch
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset
import itertools
import csv
import pandas as pd
import spacy
from torchtext.vocab import GloVe
import re
from torchtext import data
import pickle
import torch.optim as optim
import gensim
import torchtext.vocab as vocab
from tqdm import tqdm_notebook
#Verify CUDA acceleration should print cuda:0
print(device)

cuda:0


In [0]:
# Mount your google drive. 
# Use this to save your PyTorch model for submission
from google.colab import drive
drive.mount('/content/gdrive')
#Test drive access. 
#You should have a test.txt under a new folder cis530_hw6 in your Google drive
!ls

Mounted at /content/gdrive
drive  gdrive  sample_data


In [0]:
from torchtext.data import Field
import spacy
from gensim.models import KeyedVectors

# Train Word Embeddings and save
 
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' # from above
model_em = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
weights = torch.FloatTensor(model_em.vectors)
t_embedding = nn.Embedding.from_pretrained(weights)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# NLP = spacy.load('en')
MAX_CHARS = 20000
MAX_VOCAB_SIZE = 25000
comment = data.Field(
    tokenize='spacy'
)
label = data.LabelField()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def load_data(train_file, val_file, test_file, covid19_file):

    train = data.TabularDataset(
        path=train_file, format='csv', skip_header=True,
        fields=[
            ('label', label),
            ('text', comment),
        ])
    val = data.TabularDataset(
        path=val_file, format='csv', skip_header=True,
        fields=[
            ('label', label),
            ('text', comment),
        ])
    test = data.TabularDataset(
        path=test_file, format='csv', skip_header=True,
        fields=[
            ('label', label),
            ('text', comment),
        ])
    
    covid19 = data.TabularDataset(
        path=covid19_file, format='csv', skip_header=True,
        fields=[
            ('label', label),
            ('text', comment),
        ])
    
    comment.build_vocab(train, vectors=GloVe(name='6B', dim=100),
                        max_size=MAX_VOCAB_SIZE,
                        min_freq=5,
                        unk_init=torch.Tensor.normal_
                        )#
          
    label.build_vocab(train)
    print(label.vocab.stoi)

    BATCH_SIZE = 64

    train_iterator, val_iterator, test_iterator, covid19_iterator = data.BucketIterator.splits(
        (train, val, test, covid19),
        batch_size=BATCH_SIZE,
        device=device,
        sort=False)
    # pickle.dump(train, open("train_iter.p", "wb"))
    # pickle.dump(val, open("val_iter.p", "wb"))
    # pickle.dump(test, open("test_iter.p", "wb"))
    return train_iterator, val_iterator, test_iterator, covid19_iterator


class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim,
                 dropout, pad_idx):
        super().__init__()

        self.embedding =  nn.Embedding(vocab_size, embedding_dim)

        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1,
                      out_channels=n_filters,
                      kernel_size=(fs, embedding_dim))
            for fs in filter_sizes
        ])

        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        text = text.permute(1, 0)
        embedded = self.embedding(text)
        embedded = embedded.unsqueeze(1)
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        cat = self.dropout(torch.cat(pooled, dim=1))
        return self.fc(cat)

In [0]:
############################################################
# Reference Code
############################################################
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]]), max_preds, correct


def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:
        preds=[]
        trues=[]
        optimizer.zero_grad()
        predictions = model(batch.text)
        loss = criterion(predictions, batch.label)
        acc,max_preds,correct = categorical_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        # f1_score
        preds += predictions.squeeze(1).argmax(dim=1).tolist()
        trues += batch.label.tolist()

    return epoch_loss / len(iterator), epoch_acc / len(iterator), preds, trues


def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    preds=[]
    trues=[]
    model.eval()

    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text)
            loss = criterion(predictions, batch.label)
            acc, max_pred, correct = categorical_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            # f1_score
            preds += predictions.squeeze(1).argmax(dim=1).tolist()
            trues += batch.label.tolist()

    return epoch_loss / len(iterator), epoch_acc / len(iterator), preds, trues

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs




In [0]:
import os
os.listdir("./gdrive/Shared drives/cis530_final_project/MS4/data")

['positive-words.txt',
 'stopwords.txt',
 'negative-words.txt',
 'test.csv',
 'val.csv',
 'train.csv',
 '00_30_tweets.csv',
 'covid19.csv']

In [0]:
# #pd.read_csv('./gdrive/Shared drives/cis530_final_project/MS4/00_30_tweets.csv',)
# covid = data.TabularDataset(
#         './gdrive/Shared drives/cis530_final_project/MS4/00_30_tweets.csv', format='csv', skip_header=True,
#         fields=[
#             ('label', label),
#             ('text', comment),
#         ])

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
  
  train_iterator, val_iterator, test_iterator, covid19_iterator = load_data('./gdrive/Shared drives/cis530_final_project/MS4/data/train.csv', './gdrive/Shared drives/cis530_final_project/MS4/data/val.csv', 
                                                          './gdrive/Shared drives/cis530_final_project/MS4/data/test.csv','./gdrive/Shared drives/cis530_final_project/MS4/data/covid19.csv')
  INPUT_DIM = len(comment.vocab)# comment.vocab
  EMBEDDING_DIM = 300
  N_FILTERS = 300
  FILTER_SIZES = [3,4,5]
  #3,4,5
  OUTPUT_DIM = len(label.vocab)
  DROPOUT = 0.5
  PAD_IDX = comment.vocab.stoi[comment.pad_token]
  N_EPOCHS = 5

  model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
  model.embedding = t_embedding
  #UNK_IDX = comment.vocab.stoi[comment.unk_token]

  #model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
  #model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
 
  optimizer = optim.Adadelta(model.parameters(), weight_decay=1e-5) # apply l2-reg
  criterion = nn.CrossEntropyLoss()
  model = model.to(device)
  criterion = criterion.to(device)
  best_valid_loss = float('inf')
  train_preds=[]
  train_trues=[]
  val_preds =[]
  val_trues =[]
  for epoch in range(N_EPOCHS):

      start_time = time.time()
      train_loss, train_acc, train_pred, train_true = train(model, train_iterator, optimizer, criterion)
      valid_loss, valid_acc, val_pred, val_true = evaluate(model, val_iterator, criterion)
      val_preds.extend(val_pred)
      val_trues.extend(val_true)
      train_preds.extend(train_pred)
      train_trues.extend(train_true)

      end_time = time.time()

      epoch_mins, epoch_secs = epoch_time(start_time, end_time)

      if valid_loss < best_valid_loss:
          best_valid_loss = valid_loss
          torch.save(model.state_dict(), 'baseline-model.pt')

      print(f'Epoch: {epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
      print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
      print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')


.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                           
100%|█████████▉| 398188/400000 [00:17<00:00, 22135.95it/s]

defaultdict(<function _default_unk_index at 0x7faa6272fb70>, {'0': 0, '1': 1, '-1': 2})


100%|█████████▉| 398188/400000 [00:30<00:00, 22135.95it/s]

Epoch: 01 | Epoch Time: 4m 22s
	Train Loss: 0.336 | Train Acc: 88.13%
	 Val. Loss: 0.199 |  Val. Acc: 93.60%
Epoch: 02 | Epoch Time: 4m 21s
	Train Loss: 0.229 | Train Acc: 92.52%
	 Val. Loss: 0.181 |  Val. Acc: 94.19%
Epoch: 03 | Epoch Time: 4m 20s
	Train Loss: 0.200 | Train Acc: 93.54%
	 Val. Loss: 0.163 |  Val. Acc: 94.99%
Epoch: 04 | Epoch Time: 4m 18s
	Train Loss: 0.183 | Train Acc: 94.19%
	 Val. Loss: 0.162 |  Val. Acc: 95.10%
Epoch: 05 | Epoch Time: 4m 18s
	Train Loss: 0.172 | Train Acc: 94.52%
	 Val. Loss: 0.159 |  Val. Acc: 95.29%


In [0]:

test_loss, test_acc, test_preds, test_trues = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')
  
# f1 score
from sklearn.metrics import f1_score,recall_score, precision_score
#print(f1_score(preds, trues, average='weighted'))
# training 
print(f1_score(train_trues, train_preds, average='macro'))
print(precision_score(train_trues,train_preds,average='macro'))
print(recall_score(train_trues, train_preds,average='macro'))

# validation
print(f1_score(val_trues, val_preds, average='macro'))
print(precision_score(val_trues,val_preds,average='macro'))
print(recall_score(val_trues, val_preds,average='macro'))

#test 
print(f1_score(test_trues, test_preds, average='macro'))
print(precision_score(test_trues,test_preds,average='macro'))
print(recall_score(test_trues, test_preds,average='macro'))


'''
0.8812620211468666
0.8854992768977231
0.8774410774410774
0.9104255886331983
0.9304217405792423
0.8945494308819626
0.9200114337537629
0.9372097195318804
0.9060100274986641

300 5,5
0.9357373530580609

300 3,4,5
0.94
'''

Test Loss: 0.157 | Test Acc: 95.33%
0.8955047475022931
0.9013233487856095
0.8903424904147448
0.9299637694804748
0.9447862692438064
0.9174752236133417
0.9386775267449847
0.9528604694229416
0.9267162249856472


'\n0.8812620211468666\n0.8854992768977231\n0.8774410774410774\n0.9104255886331983\n0.9304217405792423\n0.8945494308819626\n0.9200114337537629\n0.9372097195318804\n0.9060100274986641\n\n300 5,5\n0.9357373530580609\n\n300 3,4,5\n0.94\n'

In [0]:
print('\n0.8812620211468666\n0.8854992768977231\n0.8774410774410774\n0.9104255886331983\n0.9304217405792423\n0.8945494308819626\n0.9200114337537629\n0.9372097195318804\n0.9060100274986641\n\n300 5,5\n0.9357373530580609\n\n300 3,4,5\n0.94\n')


0.8812620211468666
0.8854992768977231
0.8774410774410774
0.9104255886331983
0.9304217405792423
0.8945494308819626
0.9200114337537629
0.9372097195318804
0.9060100274986641

300 5,5
0.9357373530580609

300 3,4,5
0.94



In [0]:
print(len(test_preds))
print(len(test_trues))
for i in range(100):
  if test_preds[i]!=test_trues[i]:
    print(i,test_preds[i],test_trues[i])

235992
235992
0 1 0
6 0 1
10 2 0
37 1 2
49 0 2
69 0 2
83 0 1
84 0 2
94 2 0


In [0]:
covid19_loss, covid19_acc, covid19_preds, covid19_trues = evaluate(model, covid19_iterator, criterion)
print(f'Test Loss: {covid19_loss:.3f} | Test Acc: {covid19_acc * 100:.2f}%')
print(f1_score(covid19_trues, covid19_preds, average='macro'))
print(precision_score(covid19_trues,covid19_preds,average='macro'))
print(recall_score(covid19_trues, covid19_preds,average='macro'))

Test Loss: 2.332 | Test Acc: 58.65%
0.5790058252428509
0.5969053138430689
0.5720136182441239


In [0]:
for i in range(100):
  if covid19_preds[i]!=covid19_trues[i]:
    print(i,covid19_preds[i],covid19_trues[i])

6 0 2
7 2 0
8 0 2
10 2 0
11 2 0
17 2 0
19 2 0
20 2 0
26 2 0
27 1 0
29 2 1
31 2 0
32 1 0
35 0 2
39 1 0
42 2 0
44 2 1
45 2 0
47 0 1
48 0 2
49 0 2
50 0 2
52 2 1
53 1 0
54 2 0
56 2 0
61 2 0
66 2 0
68 2 0
73 2 0
76 2 0
78 1 0
81 2 0
85 2 0
95 0 2


In [0]:
df = pd.read_csv('./gdrive/Shared drives/cis530_final_project/MS4/data/test.csv','./gdrive/Shared drives/cis530_final_project/MS4/data/covid19.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [0]:
df.loc[29].values#split(',')

array(["-1,Called it. I said if they don't start winning they will have to trade people..... URL"],
      dtype=object)

In [0]:
import csv
from itertools import zip_longest
d = [test_preds]
export_data = zip_longest(*d, fillvalue = '')
with open('extension1_result.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(("label"))
      wr.writerows(export_data)
myfile.close()



In [0]:
import pandas as pd
def label_standard(file):
    values = list(pd.read_csv(file).values.T[0])
    for i in range(len(values)):
        if values[i] == 2:
            values[i] = -1
    print(set(values))
    c_df = pd.DataFrame(values)
    c_df = c_df.rename(columns={0:'label'})
    c_df.to_csv(file,index=False)
label_standard('extension1_result.csv')

{0, 1, -1}
